# ICBC 2023 Lower Mainland vehicle dataset

## Source

In [26]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

In [27]:
 # Check if file path is valid.
file_path = './icbc/Vehicle Population - 2023 Passenger Vehicles_Full _data.csv'
if not os.path.isfile(file_path):
    fnf_err = f'{file_path} not found.'
    raise FileNotFoundError(fnf_err)

vehicle_df = pd.read_csv(file_path)

# Take a peek at the first couple of rows in the dataset.
print(vehicle_df.head(2))

dataset_columns = vehicle_df.columns.tolist()
print(f"This dataset contains the following columns {dataset_columns}")

num_rows, num_cols = vehicle_df.shape
print(f"Data set shape {num_rows} rows x {num_cols} cols")

  Veh Pop - Criteria Selector Vehicle Use Anti Theft Device Indicator  \
0              Lower Mainland    Business                          No   
1              Lower Mainland    Business                          No   

             Body Style Electric_Vehicle_Indicator Fleet Vehicle Indicator  \
0         Fourdoorsedan                         No                      No   
1  Fourdoorstationwagon                         No                      No   

  Fuel Type Hybrid Vehicle Indicator        Make                Model  \
0    Diesel                       No  VOLKSWAGEN  RABBIT OTHER MODELS   
1    Diesel                       No      TOYOTA   LAND CRUISER WAGON   

   Model Year Municipality             Owner Type          Region  \
0        1978      Langley  External organization  Lower Mainland   
1        1991      Burnaby  External organization  Lower Mainland   

   Vehicle Count  
0              1  
1              1  
This dataset contains the following columns ['Veh Pop - Crit

# Initial comments
This dataset contains redundant columns. We specifically set the criteria to Lower Mainland vehicles on ICBC's data portal. We can get rid of the columns 'Veh Pop - Criteria Selector' and 'Region'.

In [28]:
vehicle_df.drop(['Veh Pop - Criteria Selector', 'Region'], axis=1, inplace=True)

dataset_columns = vehicle_df.columns.tolist()
print(f"This dataset contains the following columns {dataset_columns}")

num_rows, num_cols = vehicle_df.shape
print(f"Data set shape {num_rows} rows x {num_cols} cols")
assert num_cols == 13, "Columns delete failed."

This dataset contains the following columns ['Vehicle Use', 'Anti Theft Device Indicator', 'Body Style', 'Electric_Vehicle_Indicator', 'Fleet Vehicle Indicator', 'Fuel Type', 'Hybrid Vehicle Indicator', 'Make', 'Model', 'Model Year', 'Municipality', 'Owner Type', 'Vehicle Count']
Data set shape 719144 rows x 13 cols


# Exploratory Data Analysis
Here we take a quick look at each column to see if there's any obvious redundancies (if all values for col C are 'xyz', then there's isn't much to be investigated in col C)

In [29]:
for c in dataset_columns:
    print(f"Examining column {c}")
    # Find unique values in this column.
    uniques = vehicle_df[c].unique()
    print(len(uniques))

Examining column Vehicle Use
3
Examining column Anti Theft Device Indicator
2
Examining column Body Style
24
Examining column Electric_Vehicle_Indicator
2
Examining column Fleet Vehicle Indicator
2
Examining column Fuel Type
18
Examining column Hybrid Vehicle Indicator
2
Examining column Make
427
Examining column Model
9496
Examining column Model Year
116
Examining column Municipality
54
Examining column Owner Type
2
Examining column Vehicle Count
284


# Comments
- There are 3 categories for vehicle use. We'll have to dig into that later.
- There are 2 categories for anti theft device indicator. This is likely a Boolean field. A vehicle either has an anti theft device or it does not.
- There are 24 body styles. We'll have to dig into that later.
- There are 2 categories for electric vehicle indicator. This must be a Boolean field for obvious reasons. We'll have to check if a hybrid (Battery Electric Vehicle - BEV) and a PHEV (Plug-in Electric Vehicle) are declared as electric vehicles or not.
- There are 2 categories for fleet vehicle indicator. This is definitely a Boolean field.
- There are 18 categories for fuel type, which is more than we anticipated (diesel, gasoline, electric, LPG).
- There are 427 vehicle makes, which seems too high a number. We'll have to dig into that - are the makes misspelled or sometimes present as acronyms and sometimes in full?
- There are 9496 models. This seems slightly too many. We'll have to take a look later.
- There are 116 model years, which is unexpected. We don't expect the years to span more than 50/60 years.
- There are 54 municipalities, which is expected for the Lower Mainland.
- There are 2 types of owners.
- At most, there are 284 different values for vehicle count.

In [30]:
print(f"Unique values for vehicle use = {vehicle_df['Vehicle Use'].unique()}")
print(f"Anti Theft Device Indicator values ={vehicle_df['Anti Theft Device Indicator'].unique()}")
print(f"Body styles = {vehicle_df['Body Style'].unique()}")
print(f"Electric vehicle indicator ={vehicle_df['Electric_Vehicle_Indicator'].unique()}")
print(f"Fleet vehicle indicator = {vehicle_df['Fleet Vehicle Indicator'].unique()}")
print(f"Fuel types = {vehicle_df['Fuel Type'].unique()}")
print(f" Hybrid vehicle indicator = {vehicle_df['Hybrid Vehicle Indicator'].unique()}")

Unique values for vehicle use = ['Business' 'Other' 'Personal']
Anti Theft Device Indicator values =['No' 'Yes']
Body styles = ['Fourdoorsedan' 'Fourdoorstationwagon' 'Twodoorfastback'
 'Twodoorstationwagon' 'Dualpurpose' 'Fourdoorcoupe' 'Fourdoorfastback'
 'Hatchback' 'Twodoorconvertible' 'Twodoorcoupe' 'Twodoorhardtop'
 'Snowmobile' 'Twodoorsedan' 'Wheeledatv' 'Lowspeedvehicle'
 'Fourdoorhardtop' 'Golfcart' 'Limousinepassenger' 'Sportconvertible'
 'Threewheeled' 'Fourdoorconvertible' 'Workutilitypassengervehicle'
 'Dunebuggy' 'Amphibiousvehicle']
Electric vehicle indicator =['No' 'Yes']
Fleet vehicle indicator = ['No' 'Yes']
Fuel types = ['Diesel' 'Gasoline' 'Electric' 'Gasoline Electric' 'Multifuels' 'Butane'
 'Diesel Natural Gas' 'Gasoline Natural Gas' 'Natural Gas' 'Propane'
 'Diesel Butane' 'Gasoline Propane' 'Other' 'Diesel Propane' 'Alcohol'
 'Gasoline Alcohol' 'Hydrogen' 'Propane Natural Gas']
 Hybrid vehicle indicator = ['No' 'Yes']


### Vehicle use
### Anti theft device indicator
As we suspected, the anti theft device indicator field is a Boolean one.
### Body styles
We initially thought that there will only be a few body styles: sedan, hatchback, station wagon, coupe, convertible, pick up truck, crossover/SUV.
### Electric vehicle indicator
This is indeed a Boolean field.
### Fleet vehicle Indicator
This is also a Boolean field.
### Fuel types
There are values which were not anticipated in this column, such as 'butane' and 'propane'
### Hybrid vehicle indicator
This is indeed a Boolean field. It would be interesting to see how this column correlates with the fuel type column.

In [31]:
model_years = vehicle_df['Model Year'].unique()
oldest, newest = min(model_years), max(model_years)
print(f"oldest and newest are {oldest}, {newest}")
all_years = [_ for _ in range(oldest, newest+1)]
print(f"Model years = {model_years}")
missing_years = [_ for _ in all_years if _ not in all_years]
print(f"Missing years = {missing_years}")

oldest and newest are 1908, 2024
Model years = [1978 1991 1997 1998 2006 2011 2005 1993 1994 2015 1964 1996 1999 2001
 2003 2004 2007 2009 2010 2012 2014 2018 1995 2000 2002 2016 2017 1990
 1992 1966 1972 1981 1930 1965 1968 1986 1963 1969 1976 1980 1988 1970
 1971 1967 2013 2021 2019 2024 2023 2022 2020 2008 1989 1987 1935 1959
 1962 1979 1955 1973 1983 1950 1951 1982 1984 1985 1949 1952 1953 1956
 1957 1958 1960 1928 1954 1974 1923 1947 1977 1929 1936 1933 1932 1937
 1938 1939 1946 1948 1942 1961 1926 1975 1931 1934 1941 1927 1915 1940
 1924 1909 1913 1908 1912 1920 1916 1925 1921 1944 1911 1943 1910 1918
 1917 1922 1945 1914]
Missing years = []


In [32]:
municipalities = vehicle_df['Municipality'].unique()
print(f"municipalities = {municipalities}")

print(f"Owner type {vehicle_df['Owner Type'].unique()}")

municipalities = ['Langley' 'Burnaby' 'Vancouver' 'Port Coquitlam' 'Richmond' 'Surrey'
 'Whistler' 'North Vancouver' 'Coquitlam' 'Chilliwack' 'New Westminster'
 'Abbotsford' 'Mission' 'Halfmoon Bay' 'Bowen Island' 'Roberts Creek'
 'Gibsons' 'Ubc' 'Cultus Lake' 'Maple Ridge' 'Delta' 'Squamish'
 'Port Moody' 'Egmont' 'Pitt Meadows' 'Sechelt' 'Agassiz' 'West Vancouver'
 'Hope' 'White Rock' 'Pemberton' 'Harrison Hot Springs' 'Furry Creek'
 'Britannia Beach' 'Deroche' 'Harrison Lake' 'Chilliwack River Valley'
 'Popkum' 'Lake Errock' 'Madeira Park' 'Lions Bay' 'Dewdney' 'Anmore'
 'Hatzic' 'Belcarra' 'Pemberton Meadows' 'Yale' 'Columbia Valley'
 'Garden Bay' 'Boston Bar' 'Lindell Beach' 'Sunshine Valley' 'Ruby Lake'
 'Port Mellon']
Owner type ['External organization' 'Person']


## Data cleaning
### About Makes
We found out that there are 427 vehicle makes. This seems way to many. So let's check if there are any 

In [33]:
makes = vehicle_df['Make'].unique()
makes_lower = set(map(str.upper, makes)) # use a set to keep unique elements.
print(f"upper case make length comparison {len(makes_lower) == len(makes)}")
# converting to lower case didn't reduce the number of makes.
print(makes_lower)

upper case make length comparison True
{'KAISER', 'HYUNDAI TRUCK/VAN', 'MERCEDES-BENZ TRUCK', 'CHEVROLET TRUCK/VAN', 'EXCALIBUR', 'MORGAN', 'SKI-D00', 'CF', 'SKI=DOO', 'MOBILITY VENTURES', 'JORDAN', 'U-BUILT', 'RILEY', 'ROLLSROYCE', 'DODGE/RAM', 'KIA', 'SC CARTS', 'BRP', 'LEYLAND', 'LEXUS TRUCK/VAN', 'BOMBARDIER', 'CATERHAM', 'ALVIS', 'ARGO', 'CAN EXPLOR', 'ARCTIC', 'PONTIAC', 'TRIUMPH', 'TRAILBLAZR', 'KINGQUAD', 'POLARIS', 'EGLMOTOR', 'EDSEL', 'JINYUN COU', 'MAGNUM', 'METEOR', 'BRICKLIN', 'PIERCE', 'SUNL', 'ASUNA', 'MERCEDES-BENZ', 'SEGWAY', 'UBUILT', 'AURORA', 'BENTLEY TRUCK/VAN', 'ILTIS', 'SUBARU TRUCK/VAN', 'KUBOTA', 'JOYNER', 'JIANGSU', 'BOBCAT', 'DODGE/RAM TRUCK/VAN', 'GEO', 'JAGUAR TRUCK/VAN', 'CANAM', 'TRACKER', 'FARGO', 'SINGER', 'RAZOR', 'TRACKERPTV', 'VPG', 'DATSUN/NISSAN', 'CATERPILAR', 'VOLVO', 'FERARRI', 'TATA', 'INFINITI', 'JOY', 'A.C.', 'CADDY', 'MOBI', 'HENRY J', 'BUGATTI', 'INTERNATIONAL', 'ARCTIC FOX', 'AUDI', 'ESSEX', 'TOYOTA TRUCK/VAN', 'ROLLS ROYCE', 'PONTIAC TRUC

In [34]:
misspelt: dict = {
    'PORSHE': 'PORSCHE',
    'LAMBORGHIN': 'LAMBORGHINI',
    'TOYOYA': 'TOYOTA',
    'TOTOYA' : 'TOYOTA',
    'TOOYTA' : 'TOYOTA',
    'MECEDES': 'MERCEDES-BENZ',
    'MERCEDES' : 'MERCEDES-BENZ',
    'MERDECES' : 'MERCEDES-BENZ',
    'MERCEDSBNZ' : 'MERCEDES-BENZ',
    'YAHAMA': 'YAMAHA',
    'YAHMA' : 'YAMAHA',
    'INTERNATIO': 'INTERNATIONAL',
    'FERARRI' : 'FERRARI',
    'MITSHUBISH' : 'MITSUBISHI',
    'MITSHU' : 'MITSUBISHI',
    'CHEVEROLET' : 'CHEVROLET',
    'CHERVOLET' : 'CHEVROLET',
    'CHEV' : 'CHEVROLET',
    'CHEVY' : 'CHEVROLET',
    'DATSUN': 'NISSAN',
    'DATSUN/NISSAN' : 'NISSAN',
    'ROLLS ROYC' : 'ROLLS ROYCE', 
    'JOHNDEERE' : 'JOHN DEERE',
    'ALFA': 'ALFA ROMEO',
    'ASTON MART' : 'ASTON MARTIN',
    'TELSA': 'TESLA'
}

vehicle_df['Make'] = vehicle_df['Make'].map(misspelt).fillna(vehicle_df['Make'])
makes = vehicle_df['Make'].unique()
print(f"Cleaned makes length = {len(makes)} and {makes=}")

Cleaned makes length = 402 and makes=array(['VOLKSWAGEN', 'TOYOTA', 'FORD TRUCK/VAN', 'LAND ROVER',
       'AUDI TRUCK/VAN', 'SMART', 'MITSUBISHI', 'JEEP', 'FIAT', 'HYUNDAI',
       'HONDA', 'FORD', 'ACURA', 'PONTIAC', 'NISSAN', 'ACURA TRUCK/VAN',
       'CHEVROLET', 'KIA', 'LEXUS TRUCK/VAN', 'GMC TRUCK/VAN',
       'CHEVROLET TRUCK/VAN', 'NISSAN TRUCK/VAN', 'TOYOTA TRUCK/VAN',
       'DODGE/RAM TRUCK/VAN', 'HONDA TRUCK/VAN', 'SUBARU',
       'HYUNDAI TRUCK/VAN', 'CHRYSLER TRUCK/VAN', 'KIA TRUCK/VAN',
       'MERCEDES-BENZ TRUCK', 'CHRYSLER', 'DODGE/RAM', 'PLYMOUTH',
       'MERCEDES-BENZ', 'BMW', 'YAMAHA', 'ARCTIC CAT', 'SKI-DOO',
       'MASERATI', 'PORSCHE', 'LEXUS', 'SC CARTS', 'PORSCHE TRUCK/VAN',
       'JAGUAR', 'INFINITI', 'BMW TRUCK/VAN', 'HUMMER',
       'CADILLAC TRUCK/VAN', 'MAZDA TRUCK/VAN', 'INFINITI TRUCK/VAN',
       'LINCOLN TRUCK/VAN', 'VOLVO TRUCK/VAN', 'JAGUAR TRUCK/VAN',
       'VOLKSWAGEN TRUCK/VAN', 'AUDI', 'BUICK', 'MAZDA',
       'SUZUKI TRUCK/VAN', 'BUICK TRUC

In [35]:
"""
confusion with switching model names and brands.
'corolla': 'toyota',
'outback' : 'subaru',
'ALPHARD' : 'TOYOTA'
'isetta': 'bmw',
    'capri': 'ford',
    'tundra' : 'toyota',
    'anglia' : 'ford',
        'maverick' : 'ford',
    'edsel' : 'ford'
    'eunos' : 'mazda'
"""
"""
truckvan = brand.split()
if truckvan[1] == 'truck/van':
    brand = truckvan[0]
"""

"\ntruckvan = brand.split()\nif truckvan[1] == 'truck/van':\n    brand = truckvan[0]\n"